In [1]:
import os
import pandas as pd

# Read the data
df = pd.read_csv("data" + os.sep + "mbti_preprocessed_complete.csv")
df.head()

,Unnamed: 0,type,posts,encodedType,preprocessed_posts,extro_intro,intu_obs,feel_think,prosp_judg
0,0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,"['youtube', 'tumblr', 'enfp', 'intj', 'moment'...",0,1,1,0
1,1,ENTP,'I'm finding the lack of me in these posts ver...,3,"['im', 'finding', 'the', 'lack', 'of', 'post',...",1,1,0,1
2,2,INTP,'Good one _____ https://www.youtube.com/wat...,11,"['good', 'one', 'youtube', 'of', 'course', 'i'...",0,1,0,1
3,3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,"['dear', 'intp', 'i', 'enjoyed', 'conversation...",0,1,0,0
4,4,ENTJ,'You're fired.|||That's another silly misconce...,2,"['youre', 'fired', 'thats', 'another', 'silly'...",1,1,0,0


In [2]:
#extroverted phrases & engrams

#TODO Add Features before preprocessing

##Surface Realisation Features
#Feature Extroverts: "hi" at the beginning of a post
#Feature Introverts: "hello" at the beginning of a post
#Feature Extroverts: Multiple Exclamation Marks "*![opt. whitespace]!*" (STRONGLY CORELLATED WITH EXTROVERTED)
#Feature Introverts: Multiple full stops "*.[opt. whitespace].*"

#With Rel.F Ratio > 1.0


#Features bigrams for Extroverts
extroverted_bigrams = [

    "looking forward",
    "it was",
    "next week",
    "a bit",
    "up with",
    "will be",
    "i was",
    "at the",
    "to see",
    "which is",
    "for a",
    "i have",
    "ill be",
    "a good",
    "we went",
    "want to",
    "need to",
    "able to"
]

#Features stylistic_impressions
extroverted_stylistic_impressions = [
    "catch up",
    "take care"
]

#Feature Quantifiers (add them to one)
#Introverts generally tend to show a preference for a greater use of quantifiers 
interoverted_quantifiers_tendency = [
    "a lot",
    "a few",
]

#Feature_Quantifiers
#For those uniquely way higher correlation with introvert
introverted_quantifiers_uniquely = [
    "all the",
    "one of",
    "lots of",
    "loads of",    
]

#Feature First_Person_singular Pronom
#Introverts make extensive use of first person singular pronoun
introverted_first_person_singular_pronoun = [
    "i can",
    "ive got",
    "ive done",
    "i think",
    "i am",
    "i",
]

#Feature: NegationCount
#Negations are more common in introverts
introverted_negations = [
    "dont",
    "cant"
    "wont"
]

#Feature: Word Count per Post. Extroverts tend to write longer posts

#Message Introverts show perefernce for coordinating conjunctions "and" and "but" whilst
#Extraverts uniquely show use of the ksubordinating "which"

#Feature ordinating conjuctions (and, but)
#Feature subordinating conjuctions(which)